In [1]:
import pandas as pd
import datasets
from datasets import Dataset
from datasets import load_dataset
from transformers import LongformerTokenizerFast, LongformerForSequenceClassification, Trainer, TrainingArguments, LongformerConfig
import torch.nn as nn
import torch
from torch.utils.data import Dataset, DataLoader
import numpy as np
from sklearn.metrics import accuracy_score, precision_recall_fscore_support
from tqdm import tqdm
#import wandb
import os

In [2]:
#train_df = pd.read_csv('./v1/articles_train.tsv', index_col=False, sep='\t')
#validate_df = pd.read_csv('./v1/articles_validate.tsv', index_col=False, sep='\t')



In [3]:
### convert pandas df (only columns 'titletext' and 'label') to nlp Dataset
#train_data = Dataset.from_pandas(train_df[['context','label']])
#validation_data = Dataset.from_pandas(validate_df[['context','label']])

In [4]:
_features=datasets.Features(
    {
        "content": datasets.Value("string"),
        "label": datasets.Value("int64"),
        "polarity": datasets.ClassLabel(num_classes=5, names=['Left','LeanLeft','Center','LeanRight','Right'])
        
    }
)

dataset = load_dataset('csv', data_files={
    "train": './v1/articles_train.tsv',
    "validation": './v1/articles_validate.tsv',
    "test": './v1/articles_test.tsv',
}, delimiter="\t")

# ['train', 'test']

Using custom data configuration default-1c361d7acb47da84
Reusing dataset csv (/home/bhakthi/.cache/huggingface/datasets/csv/default-1c361d7acb47da84/0.0.0/9144e0a4e8435090117cea53e6c7537173ef2304525df4a077c435d8ee7828ff)


  0%|          | 0/3 [00:00<?, ?it/s]

In [5]:
dataset['train'][10]['polarity']

'Right'

In [6]:
### https://github-dotcom.gateway.web.tr/huggingface/datasets/issues/2365
dataset = dataset.map(_features.encode_example, features=_features)
#dataset['train'].cast_(_features)
#print(dataset.keys())


Loading cached processed dataset at /home/bhakthi/.cache/huggingface/datasets/csv/default-1c361d7acb47da84/0.0.0/9144e0a4e8435090117cea53e6c7537173ef2304525df4a077c435d8ee7828ff/cache-d1cfad0eb3e59adc.arrow
Loading cached processed dataset at /home/bhakthi/.cache/huggingface/datasets/csv/default-1c361d7acb47da84/0.0.0/9144e0a4e8435090117cea53e6c7537173ef2304525df4a077c435d8ee7828ff/cache-058cc024f1c38e2e.arrow
Loading cached processed dataset at /home/bhakthi/.cache/huggingface/datasets/csv/default-1c361d7acb47da84/0.0.0/9144e0a4e8435090117cea53e6c7537173ef2304525df4a077c435d8ee7828ff/cache-b2b69b1c7176c7da.arrow


In [7]:
dataset['train'][10]['polarity']

4

In [8]:
#dataset.cast_(_features)

In [9]:
dataset

DatasetDict({
    train: Dataset({
        features: ['content', 'label', 'polarity'],
        num_rows: 310553
    })
    validation: Dataset({
        features: ['content', 'label', 'polarity'],
        num_rows: 69875
    })
    test: Dataset({
        features: ['content', 'label', 'polarity'],
        num_rows: 7764
    })
})

In [10]:
train_data = dataset['train']
validation_data = dataset['validation']
test_data = dataset['test']

In [11]:
train_data[10]['polarity']

4

In [12]:
# load model and tokenizer and define length of the text sequence
#model = LongformerForSequenceClassification.from_pretrained('allenai/longformer-base-4096',
#                                                           gradient_checkpointing=False,
#                                                          attention_window = 512)
tokenizer = LongformerTokenizerFast.from_pretrained('allenai/longformer-base-4096', max_length = 2048)

In [13]:
label_map = {'Right':0,'LeanRight':1,'Center':2,'LeanLeft':3,'Left':4}
def convert_to_features(example):
    # Tokenize contexts and questions (as pairs of inputs)
    encodings = tokenizer.encode_plus(example['content'], pad_to_max_length=True, max_length=2048,
                                           add_special_tokens=True,
                                            return_token_type_ids=False,
                                            return_attention_mask=True,
                                            padding='max_length', truncation=True,
                                           )
    
    #label = torch.tensor(polarity,dtype=torch.int64)
    #encodings.update({'label': label,
    #                  'attention_mask': encodings['attention_mask']})
    return encodings

### We will sample this data for testing purpose only

## Take subset of data
train_size = 20000
val_size = 5000
import numpy as np
train_indices = np.random.randint(0, len(dataset['train']), train_size)
val_indices = np.random.randint(0, len(dataset['validation']), val_size)
train_data = dataset['train'].select(train_indices)
validation_dataset = dataset['validation'].select(val_indices)

In [14]:
train_data =  train_data.map(convert_to_features, load_from_cache_file=False)
validation_data =  validation_data.map(convert_to_features, load_from_cache_file=False)


  0%|          | 0/310553 [00:00<?, ?ex/s]

  0%|          | 0/69875 [00:00<?, ?ex/s]

In [15]:
test_data =  test_data.map(convert_to_features, load_from_cache_file=False)

  0%|          | 0/7764 [00:00<?, ?ex/s]

In [16]:
columns = ['input_ids', 'attention_mask', 'label']
train_data.set_format(type='torch', columns=columns)
validation_data.set_format(type='torch', columns=columns)
test_data.set_format(type='torch', columns=columns)


In [18]:
torch.save(train_data, './v1/train_data.pt')
torch.save(validation_data, './v1/validation_data.pt')
torch.save(test_data, './v1/test_data.pt')

In [20]:
train_data[10]

{'label': tensor(4),
 'input_ids': tensor([   0, 6460,   70,  ...,    1,    1,    1]),
 'attention_mask': tensor([1, 1, 1,  ..., 0, 0, 0])}

In [22]:
tokenizer.decode(train_data[10]['input_ids'])

'<s>get all the latest news on coronavirus and more delivered daily to your inbox sign up here we brought you fresh reporting thursday night about the origins of the deadly coronavirus pandemic now we have more we spent much of the day speaking to highly informed officials in the yous government as well as seasoned specialists on china first many in the intelligence world with experience in china suspected right away that the story the chinese government was telling about this virus was almost certainly a lie the first indication of that from official sources in beijing bret baier says more and more dots point to wuhan lab as origin of coronavirus pandemic initially chinese officials claimed the virus had jumped from an obscure scaly animal called a pangolin which was sold in the wuhan wet market that explanation didn t make sense wet markets are seafood markets pangolins are mammals so are bats which were also suspected as the source of the coronavirus in the face of skepticism the ch